In [17]:
import spacy
from spacy_layout import spaCyLayout
import re
from spacy.tokens import Doc

# Load layout parser
nlp_layout = spacy.load("en_core_web_lg")
layout = spaCyLayout(nlp_layout)

# Extract markdown from PDF
doc_pdf = layout('21794875.pdf')
raw = doc_pdf._.markdown

# Prepare cleaning pipeline
if not Doc.has_extension("markdown"):
    Doc.set_extension("markdown", default="")

nlp_clean = spacy.blank("en")

def clean_markdown(text: str) -> str:
    text = re.sub(r"<!--\s*image\s*-->", "", text, flags=re.IGNORECASE)
    text = text.replace("�", " ")
    text = text.replace("·", "- ")
    text = text.replace("•", "- ")
    text = text.replace("○", "- ")
    text = re.sub(r"[^\x09\x0A\x0D\x20-\x7E]", " ", text)
    text = re.sub(r"[ ]{2,}", " ", text)
    text = text.replace("\\n", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]+\n", "\n", text)
    return text.strip()

def preprocess_cv_to_markdown(raw_text: str):
    doc = nlp_clean(raw_text)
    cleaned = clean_markdown(raw_text)
    doc._.markdown = cleaned
    return doc

# RUN
cleaned_doc = preprocess_cv_to_markdown(raw)
print(cleaned_doc._.markdown)   # <=== IMPORTANT

2025-12-05 00:54:50,886 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-05 00:54:50,890 - INFO - Going to convert document batch...
2025-12-05 00:54:50,892 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-12-05 00:54:50,893 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-05 00:54:50,895 - INFO - easyocr cannot be used because it is not installed.
2025-12-05 00:54:50,896 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-05 00:54:50,919 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-05 00:54:50,936 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\danua\Desktop\project\playground\talentiq-env\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-05 00:54:50,937 [RapidOCR] torch.py:54: Using C:\Users\danua\Desktop\project\playground\talentiq-env\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-05 00:54:51

## ACCOUNTANT Professional Sum m ary

Bili ngual , accounti ng professional with additional expertise in autom ated system s and general ledger. An analytical, detail-oriented, organized planner who perform s well in a project setting and who com m unicates effectively with all levels of m anagem ent. Additionally has past experience in pubic accounting.

## Education

Masters , Accounting University of Florida City , State , US Bachelor , Business Adm inistration Florida International University City , State , US Associate of Arts , Liberal Arts Miam i Dade College City , State , USA Certifications

CPA, State of Florida

## Skill Highlights

- Language and Writing Skills: Bili ngual Spanish (fl uent; speak, read, and write)
- Computer Skills: Microsoft Office including, Word, Excel, Power Point, Outlook, Publisher; QuickBooks, Tax software

Professional Experience Accountant Apr 2012 to Current Company Name City , State

- Reconciled various general ledger fund accounts and prepared 

In [28]:
### Estimated token count ###
word_count = len(cleaned_doc._.markdown.split())
print("estimated token count: ", int(word_count * 1.3))    
print(cleaned_doc._.markdown)

estimated token count:  1186
## ACCOUNTANT Professional Sum m ary

Bili ngual , accounti ng professional with additional expertise in autom ated system s and general ledger. An analytical, detail-oriented, organized planner who perform s well in a project setting and who com m unicates effectively with all levels of m anagem ent. Additionally has past experience in pubic accounting.

## Education

Masters , Accounting University of Florida City , State , US Bachelor , Business Adm inistration Florida International University City , State , US Associate of Arts , Liberal Arts Miam i Dade College City , State , USA Certifications

CPA, State of Florida

## Skill Highlights

- Language and Writing Skills: Bili ngual Spanish (fl uent; speak, read, and write)
- Computer Skills: Microsoft Office including, Word, Excel, Power Point, Outlook, Publisher; QuickBooks, Tax software

Professional Experience Accountant Apr 2012 to Current Company Name City , State

- Reconciled various general ledge

In [19]:
def chunk_text(text, size=15000, overlap=400):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + size, len(text))
        chunks.append(text[start:end])
        start += size - overlap
    return chunks

In [20]:
chunks = chunk_text(cleaned_doc._.markdown)
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk)
    print()

--- Chunk 1 ---
## ACCOUNTANT Professional Sum m ary

Bili ngual , accounti ng professional with additional expertise in autom ated system s and general ledger. An analytical, detail-oriented, organized planner who perform s well in a project setting and who com m unicates effectively with all levels of m anagem ent. Additionally has past experience in pubic accounting.

## Education

Masters , Accounting University of Florida City , State , US Bachelor , Business Adm inistration Florida International University City , State , US Associate of Arts , Liberal Arts Miam i Dade College City , State , USA Certifications

CPA, State of Florida

## Skill Highlights

- Language and Writing Skills: Bili ngual Spanish (fl uent; speak, read, and write)
- Computer Skills: Microsoft Office including, Word, Excel, Power Point, Outlook, Publisher; QuickBooks, Tax software

Professional Experience Accountant Apr 2012 to Current Company Name City , State

- Reconciled various general ledger fund accoun

In [29]:
import re
import json
from json_repair import repair_json

DEFAULT_SCHEMA = {
    "summary": {
        "value": "",
        "confidence": 0.00
    },
    "education": {
        "items": [
            {
                "degree": "",
                "graduation_year": "",
                "institution": "",
                "gpa": ""
            }
        ],
        "confidence": 0.00
    },
    "work_experience": {
        "items": [
            {
                "role": "",
                "company": "",
                "description": "",
                "years": ""
            }
        ],
        "confidence": 0.00
    },
    "projects": {
        "items": [
            {
                "title": "",
                "role": "",
                "year": "",
                "description": "",
                "tech_stack": []
            }
        ],
        "confidence": 0.00
    },
    "certifications": {
        "items": [
            {
                "name": "",
                "issuer": "",
                "year": ""
            }
        ],
        "confidence": 0.00
    },
    "skills": {
        "items": [],
        "confidence": 0.00
    }
}


In [30]:
import requests
import json
import re

try:
    from json_repair import repair_json
except ImportError:
    print("'json-repair' not found")
    def repair_json(json_string):
        return json_string


def extract_from_chunk(api_url, chunk):
    prompt = f"""
    You are a resume text classifier. Extract information from the resume and return valid JSON.
    Follow EXACTLY the schema below:

    {json.dumps(DEFAULT_SCHEMA, indent=2)}

    STRICT RULES:
    1. You must classify text based on meaning, NOT position.
    2. If a section is missing or not explicitly stated, return an empty list/string AND set confidence to 0.00.
    3. Work Experience must include ONLY real employment (full-time, part-time, internship). 
    Do NOT include project-based roles, student projects, coursework, or workshop activities.
    4. Projects must include ONLY actual projects.
    5. Education must include ONLY degrees, graduation years, and institutions.
    6. Certifications must follow strict definitions:
    - "name": the certification title only
    - "issuer":  must be an organization/company/school. NEVER a program name, course name, role, or year.
    - "year": must be a 4-digit year ONLY
    7. Skills must ONLY contain a list of strings. 
    Every skill, sub-skill, method, library, framework, algorithm, and tool MUST be extracted INDIVIDUALLY.
    Example:
    Input: "Machine Learning: regression, classification"
    Output: ["Machine Learning", "Regression", "Classification"].
    DO NOT combine items. DO NOT summarize. DO NOT skip any item.
    8. Do not introduce objects inside the skills list. Only strings are allowed.
    9. Dates must be normalized to "YYYY", "YYYY-MM", or "YYYY-MM-DD".
    10. Never invent or infer missing information.
    11. Use the schema without modification.

    CONFIDENCE RULES (non-categorical):
    - Confidence must be a floating-point number between 0.00 and 1.00 (two decimals).
    - 0.80–1.00 = strongly supported, explicit, high certainty.
    - 0.40–0.79 = partially supported or incomplete evidence.
    - 0.01–0.39 = very weak evidence.
    - 0.00 = section completely absent.

    Now extract the content strictly using the provided schema from this CV chunk:

    {chunk}

    Output only VALID JSON. No text before or after.
    """

    # Calling API from cloud ngrok server google colab
    try:
        payload = {
            "messages": [
                {"role": "system", "content": "You are a resume parser that outputs only valid JSON without any explanations or code."},
                {"role": "user", "content": prompt},
            ]
        }

        # endpoint request
        with requests.post(f"{api_url}/chat/stream", json=payload, stream=True, timeout=300) as response:
            response.raise_for_status()

            # streaming response
            response_text = ""
            print("Respons dari model (streaming):")
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8')
                    if decoded_line.startswith("data: "):
                        json_data = decoded_line[len("data: "):]
                        data = json.loads(json_data)
                        content = data.get('content', '')
                        if content:
                            response_text += content
                            print(content, end="", flush=True)
            print("\n--- Streaming selesai, memproses JSON ---\n")

    except requests.exceptions.RequestException as e:
        print(f"Error saat menghubungi API: {e}")
        return json.loads(json.dumps(DEFAULT_SCHEMA))
    
    cleaned = response_text.replace("```json", "").replace("```", "").strip()
    match = re.search(r"\{[\s\S]*\}", cleaned)
    raw_json = match.group(0) if match else cleaned

    try:
        result = json.loads(raw_json)
    except:
        try:
            repaired = repair_json(raw_json)
            result = json.loads(repaired)
        except:
            return json.loads(json.dumps(DEFAULT_SCHEMA)) # deepcopy fallback

    # Ensure schema completeness
    final = json.loads(json.dumps(DEFAULT_SCHEMA))
    
    # Helper function to recursively clean and validate schema
    def clean_and_validate(obj, schema):
        if isinstance(obj, dict) and isinstance(schema, dict):
            result = {}
            for key, value in schema.items():
                if key in obj:
                    result[key] = clean_and_validate(obj[key], value)
                else:
                    result[key] = json.loads(json.dumps(value))  # Deep copy default value
            return result
        elif isinstance(obj, list) and isinstance(schema, list):
            if obj and schema:
                item_schema = schema[0]
                return [clean_and_validate(item, item_schema) for item in obj]
            return obj
        else:
            if isinstance(obj, str):
                return obj.replace("\u2022", "-").strip()
            return obj
    
    final = clean_and_validate(result, DEFAULT_SCHEMA)
    
    def validate_confidence(obj):
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == "confidence" and isinstance(value, (int, float)):
                    obj[key] = round(max(0.00, min(1.00, float(value))), 2)
                else:
                    validate_confidence(value)
        elif isinstance(obj, list):
            for item in obj:
                validate_confidence(item)
    
    validate_confidence(final)
    
    return final

In [23]:
def merge_schema(base, new):
    """Merge structured chunk results into base schema."""
    if new.get("summary", {}).get("confidence", 0) > base.get("summary", {}).get("confidence", 0):
        base["summary"] = new.get("summary", base["summary"])
        
    # # Seniority: pick first
    # if not base["seniority"] and new.get("seniority"):
    #     base["seniority"] = new["seniority"]
        
    # Merge lists from sections with items
    for key in ["education", "work_experience", "certifications", "projects"]:
        if key in new and "items" in new[key] and isinstance(new[key]["items"], list):
            if new[key]["items"]:
                base[key]["items"].extend(new[key]["items"])
                if new[key].get("confidence", 0) > base[key].get("confidence", 0):
                    base[key]["confidence"] = new[key]["confidence"]

    # Skills: dedupe and merge
    if "skills" in new and "items" in new["skills"] and isinstance(new["skills"]["items"], list):
        combined_skills = list(set(base["skills"]["items"] + new["skills"]["items"]))
        base["skills"]["items"] = combined_skills
        if new["skills"].get("confidence", 0) > base["skills"].get("confidence", 0):
            base["skills"]["confidence"] = new["skills"]["confidence"]

    return base

In [24]:
def extract_resume_batch(llm, text_chunks):
    merged = json.loads(json.dumps(DEFAULT_SCHEMA))  # deep copy

    for chunk in text_chunks:
        partial = extract_from_chunk(llm, chunk)
        merged = merge_schema(merged, partial)

    return merged


In [25]:
stage_1 = extract_resume_batch("https://76348a27f868.ngrok-free.app/", chunks)

Respons dari model (streaming):
{
  "summary": {
    "value": "Bilingual, accounting professional with additional expertise in automated systems and general ledger. An analytical, detail-oriented, organized planner who performs well in a project setting and who communicates effectively with all levels of management.",
    "confidence": 0.99
  },
  "education": {
    "items": [
      {
        "degree": "Masters",
        "graduation_year": "US",
        "institution": "University of Florida",
        "gpa": ""
      },
      {
        "degree": "Bachelor",
        "graduation_year": "US",
        "institution": "Florida International University",
        "gpa": ""
      },
      {
        "degree": "Associate of Arts",
        "graduation_year": "USA",
        "institution": "Miam i Dade College",
        "gpa": ""
      }
    ],
    "confidence": 0.99
  },
  "work_experience": {
    "items": [
      {
        "role": "Accountant",
        "company": "Company Name",
        "descriptio

# CLOUD API INFERENCE TEST

In [31]:
import requests
import json

NGROK_URL = "https://76348a27f868.ngrok-free.app/" 
API_ENDPOINT = f"{NGROK_URL}/chat/stream"

payload = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
}

print(f"Sending request to: {API_ENDPOINT}")
print("Response from model (streaming):")

try:
    with requests.post(API_ENDPOINT, json=payload, stream=True) as response:
        if response.status_code == 200:
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8')
                    if decoded_line.startswith("data: "):
                        json_data = decoded_line[len("data: "):]
                        data = json.loads(json_data)
                        print(data['content'], end='', flush=True)
            print("\n\n--- Streaming End ---")
        else:
            print(f"{response.status_code}")
            print(response.text)

except requests.exceptions.RequestException as e:
    print(f"Error connecting to server: {e}")


Sending request to: https://76348a27f868.ngrok-free.app//chat/stream
Response from model (streaming):
Hello! How can I assist you today?

--- Streaming End ---
